# Model 8: Blending inheritance

original: https://bookdown.org/amesoudi/ABMtutorial_bookdown/model8.html

- 昔々、ダーウィンが種の起源を書いた頃、生物は両親の特徴をブレンドした特徴を持って生まれてくると考えられてきた (混合遺伝)。でも、だとすると、世代を重ねるにつれて、生物の多様性は失われ、みんな同じような特性を持つ生物ばかりになってしまうのではないか？こういった謎があった。
- この謎は、メンデルの有名なエンドウ豆の実験で解明された。遺伝は、液体が混じり合うようにして起こるのではなく、離散的な粒子が混じり合うようにして起こるのだ。
- 紫の花と白い花を親として持つ子は、薄紫になるのではなく、紫か白のどちらかの花をつける。人間の身長の様に、複雑な特性は、一見、混合遺伝の様に見えるが、実際には粒子遺伝である。
- 文化進化が、混合遺伝で起こるのか粒子遺伝で起こるのか、はっきりしたことは分かっていない。
- ここでは、文化的継承が混合的に起こるとした場合のモデルを検討する。
- 文化的継承が混合的に起こるとしても、他の仕組みによって文化的多様性が維持される可能性はある。

## Model 8a: Blending inheritance

In [1]:
include("src/Agent.jl")
include("src/Output.jl")
using .Agent
using .Output

using DataFrames
using Distributions
using Plots

In [2]:
function blending_inheritance(; N::Int, n::Int, r_max::Int, t_max::Int, e::Float64 = 0.0)
    output_df = make_output_df(r_max, t_max)
    rename!(output_df, [:p => :mean, :q => :var])
    
    for r = 1:r_max
        traits = reshape(rand(Normal(), N), 1, N)

        for t = 1:t_max
            if t > 1
                traits = next_traits(traits, N, n, e)
            end

            row_num = t + (r - 1) * t_max
            output_df.r[row_num] = r
            output_df.t[row_num] = t
            output_df.mean[row_num] = mean(traits)
            output_df.var[row_num] = var(traits)
        end
    end
    
    # plot
    mean_plot = plot(
        output_df[output_df.r.== 1, :].mean,
        ylims = (-1.05, 1.05),
        title = "N = $N, n = $n",
        xlabel = "generation",
        ylabel = "trait mean",
        legend = false
    )
    for r = 2:maximum(output_df.r)
        plot!(output_df[output_df.r.== r, :].mean)
    end

    var_plot = plot(
        output_df[output_df.r.== 1, :].var,
        ylims = (-0.03, 1.03),
        title = "N = $N, n = $n",
        xlabel = "generation",
        ylabel = "trait variance",
        legend = false
    )
    for r = 2:maximum(output_df.r)
        plot!(output_df[output_df.r.== r, :].var)
    end

    plot(
        mean_plot,
        var_plot
    )
end

function next_traits(traits::Matrix{Float64}, N::Int, n::Int, e::Float64)::Matrix{Float64}
    _traits = rand(traits, N, n)

    # blending inheritance
    mean(_traits, dims = 2)
end

next_traits (generic function with 1 method)

In [ ]:
blending_inheritance(N = 10^3, n = 5, r_max = 5, t_max = 20)

In [ ]:
blending_inheritance(N = 10^3, n = 2, r_max = 5, t_max = 20)

## Model 8b: Blending inheritance with mutation

- 当然ながら、Mutation や Selection を導入すれば文化的多様性はある程度維持される。
- よって、Blending と文化的多様性は矛盾しない。

In [ ]:
function next_traits(traits::Matrix{Float64}, N::Int, n::Int, e::Float64)::Matrix{Float64}
    _traits = rand(traits, N, n)

    # mutation
    _traits = [rand(Normal(t, sqrt(e))) for t in _traits]

    # blending inheritance
    mean(_traits, dims = 2)
end

In [ ]:
blending_inheritance(N = 10^3, n = 5, r_max = 5, t_max = 20, e = 0.5)

In [ ]:
blending_inheritance(N = 10^3, n = 5, r_max = 5, t_max = 20, e = 2.0)

## Analytic Appendix

- $V$: Variance
- $e$: Error rate
- $\bar{E}$: Average of $e$

$V' = \frac{V + \bar{E}}{n}$

When $V' = V$:

$\hat{V} = \frac{e}{n-1}$